# Price based policy setup

## Setting price path

### Define values

Here we are interested in price-based policy. Note that each model has price-based policy examples applying single carbon tax profile to all region by default. However, different models has different tax profiles and possibly what kind of carbon (FOS vs. TOT) is accounted. To make a fair comparison, we need to set a consistent price path and make same carbon accounting.

In [7]:
import pandas as pd
import pandas.io.excel
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pytz
import os
import patsy #for spline regression
import scipy #for non-negative least square 
import scipy as sp 
from scipy import stats
from scipy.optimize import nnls
from numpy.linalg import inv #for matrix and statistics
import scipy as sp
import math
import statsmodels.api as sm
%matplotlib inline

First, we set consistent carbon tax in unit 2005 dollars per ton CO2. We can set exponential carbon tax path by setting the first carbon tax equals to 10 $2005 per CO2 ton in 2020, and increases by factor 1.04 each year. Perturbation is used to find elasticity.

In [8]:
p_ini = 10.0
r = 1.04
startyear = 2020
perturbation = 1.00
exponential_price = pd.DataFrame(columns=['year','$2005 per CO2 ton']) 
exponential_price['year'] =  [startyear+5*i for i in range(0,17)]
exponential_price['$2005 per CO2 ton'] =  [p_ini*perturbation*np.power(r,(i*5)) for i in range(0,17)]

One may construct another price path, say, step price. We need a value of step price and the year where step occurs. We are interested in step prices of 50, 100, and 200 dollars at year 2050 (We have looked at 400 dollars and it starts hitting corner solution). Here is an example.

In [19]:
p_ini = 200.0 #50.0 200.0
r = 1
startyear = 2020
stepyear = 2050
perturbation = 1.00
step_price = pd.DataFrame(columns=['year','$2005 per CO2 ton']) 
step_price['year'] =  [startyear+5*i for i in range(0,17)]
step_price['$2005 per CO2 ton'] = (step_price.year>=stepyear)*p_ini*perturbation*np.power(r,(i*5))

For EPPA, set price, namely $\texttt{p_ini}$ in .cas file needs conversion: $\text{set price} = \text{\$2005 per CO2 ton price}*0.84628/12*44/100$.

For GCAM, set price is in 1990 US dollars per C ton in 'Main_User_Workspace/input/policy/carbon_tax_10_5.xml' file and it needs conversion: $\text{set price} = \text{\$2005 per CO2 ton price}/12*44*(0.72262)$.

For MERGE, set price, namely $\texttt{ametax}$ in 'm7.gms' file is in 2005 dollars per CO2 ton, so we do not need any further conversion. However, to be precise we should change  $\text{\$2005 per CO2 ton price} = \texttt{3.67*ametax/1.135}$ to be $\text{\$2005 per CO2 ton price} = \texttt{3.67*ametax*0.88723}$ for correct precision. [In detail,  $\text{\$2005 per CO2 ton price} = \texttt{3.67*ametax/1.135}$$*12/44/0.88723 \approx \texttt{ametax}$. (3.67 comes from 44/12 CO2-C conversion and 1.135 is likely from deflation: 1/0.88723 = 1.127) so all factors cancel out.] 

We can define function called 'price_path_conversion' to generate conversion for any price path:

In [10]:
def price_path_conversion(pricetable):
    pricetable['MERGE set price'] = pricetable['$2005 per CO2 ton']*1
    pricetable['EPPA set price'] = pricetable['$2005 per CO2 ton']*0.84628/12*44/100
    pricetable['GCAM set price'] = pricetable['$2005 per CO2 ton']/12*44*0.72262
    return pricetable;

In [11]:
#price_path_conversion(exponential_price)

In [20]:
price_path_conversion(step_price)

,year,$2005 per CO2 ton,MERGE set price,EPPA set price,GCAM set price
0,2020,0,0,0.000000,0.000000
1,2025,0,0,0.000000,0.000000
2,2030,0,0,0.000000,0.000000
3,2035,0,0,0.000000,0.000000
4,2040,0,0,0.000000,0.000000
5,2045,0,0,0.000000,0.000000
6,2050,200,200,6.206053,529.921333
7,2055,200,200,6.206053,529.921333
8,2060,200,200,6.206053,529.921333
9,2065,200,200,6.206053,529.921333


In [1]:
529.921333*0.85

450.43313305

### How to modify price paths in models with perturbation

$\underline{\textbf{WARNING}}$ Some of modifications here is a dirty way. Don't forget to copy original file and name it, say, '[original name]_original' before doing any change.

Note: we are only interested in a step price path here.

$\underline{\text{EPPA}}$:
1. In price-based policy .cas file, define extra parameter called $\texttt{perturb}$ in order to vary price path and measure elasticity. Here we use $\texttt{perturb}$ from 0.85 to 1.15 with step size 0.05. Then assign $\texttt{p_ini = p_ini*perturb;}$ Also, change starting year to be 2050 ($\texttt{det_pr = 12}$). Save .cas file with name $\texttt{test_policy_price_[step price name]_[perturb].cas}$.
2. In $\texttt{eppaloop2.gms}$, go to the last line assigning value to the carbon price $\texttt{pcarblag}$. Then add the following lines:
$\texttt{pcarblag = 0;}$.
$\texttt{pcarblag\$(ord(T)>= det_pr) = p_ini;}$. We save this file as, for example, $\texttt{eppaloop2_for-price-based.gms}$
3. To run multiple cases with different values of perturbation, create a batch file, for example, $\texttt{batch_price-based.dat}$ Each scenario should run: $\texttt{call rrun [.cas file name without .cas]}$. 

$\underline{\text{GCAM}}$:

1. Follow an instruction of running reference case.

2. Start from price-based policy .xml file in $\texttt{/input/policy}$ folder, for example, $\texttt{carbon_tax_25_5.xml}$. Add lines of fixed tax for individual year: $\texttt{<fixedTax year="2050">[tax-value]</fixedTax>}$. This tax value should include perturbation. Don't forget to save this new input policy file with a distinct name.

3. Start from original $\texttt{configuration.xml}$ in exe folder. Modify it as follow:

    -Add policy in ScenarioComponents object as: $\texttt{<Value name = "policy">../input/policy/[input-policy-name].xml</Value>}$ 
    
    -Change scenarioName in Strings object as: $\quad\texttt{<Value name="scenarioName">[input-policy-name]</Value>}$. This will help us find scenarios easier in Java interface.

    -Change CalibrationActive in Bools object to be 0 in order to prevent calibration change.

4. To run multiple cases with different values of perturbation, create a batch file, for example, $\texttt{batch_price-based.dat}$. Each scenario should run $\texttt{Objects-Main}$.



$\underline{\text{MERGE}}$: 
1. Add extra price path as follow:

    - In studies.xls, there are lexicon called $\texttt{ametax}$ with values ametax10, ametax30, ametax50 in sheet named Lexicon. Add extra values 'step050','step100', and 'step200' to represent step price path. 
    
    - In m7.gms, there is a table call ametax. Add an extra line to represent a price path for each path name with MERGE set price. Note that GAMS is sensitive to tabs and spaces, so make sure all items are properly aligned.

2. Add perturbation as follow:
    -In m7.gms define extra parameter called $\texttt{perturb}$ in order to vary price path and measure elasticity. perturb ranges from 0.85 to 1.15 with step size 0.05. 
    
    -In studies.xls, go to Lexicon sheet and add new lexicon $\texttt{perturb}$ with relevant information, then go to aero sheet and add extra column for $\texttt{perturb}$ lexicon.
    
    - In m7.gms add extra line after assigning ghgtax to perturb the tax as follow:
$\texttt{\$if set perturb ghgtax("co2",tp,rg,sw) = ghgtax("co2",tp,rg,sw)* perturb("%perturb%");}$

4. To run multiple cases with different values of perturbation, add multiple lines in studies.xls modifying from the baseline case. 


## Carbon accounting issue: fossil fuel only (FOS), fossil and industry (FI), and total (TOT)

Here different IAMs may account carbon in tax calculation differently.

In MERGE by default, all carbon (TOT) is accounted in tax calculation even though it is divided into $\texttt{CCEM}$ and $\texttt{CLEV}$. After code modification, we can target in all three levels. However, since land-use (biomass) emission is relatively small in MERGE. The scenario TOT and FI should be equivalent. Here we will run accounting for FOS and TOT only to be consistent with other models.

In EPPA by default, only fossil fuel carbon (FOS) is accounted in tax calculation. One may change how carbon is targeted by using $\texttt{cflag}$ as appeared in .cas file. If the flag is 1, it will take account of all carbon (TOT). Here we will run both accounting scenarios.

In GCAM by default, carbon by subsectors are used is accounted in tax calculation. This is not equivalent to FOS, FI or TOT. Fortulately, one can target tax calculation to each subsector individually. In addition, one can include carbon emission of land use change outside subsectors using appropriate policy file. See more detail below.

### How to enforce different CO2 emissions accounting procedures (quantity-based policy included as well)

$\underline{\textbf{WARNING}}$ Some of modifications here is a dirty way. Don't forget to copy original file and name it, say, '[original name]_original' before doing any change.

$\underline{\text{EPPA}}$:

As addressed earlier, in EPPA, there are built-in options to choose different CO2 emissions accountings by changing a single parameter in a supplementary file. In $\texttt{eppa4-1/parameters/eppacoef.gms}$, there is a parameter called $\texttt{cflag(r)}$. One may assign it to be zero for CO2 emissions from fossil fuels (default) and one for total CO2 emissions. 

Extra note: Previously, Henry from EPPA team said that there is no total carbon emission counterpart in tax calculation since carbon emission from land-use is calculated as an extra. However, it can be done in eppa4-1 version. 

$\underline{\text{GCAM}}$: 

As addressed earlier, in GCAM available options, CO2 emissions accounting procedure does not match either of two procedures we want to study. In order to include CO2 emissions from fossil fuels in account of CO2 policy, one needs to construct a new policy xml tag, for example, <ghgpolicy name="CO2_FOS"> and add them after all fossil fuel technologies. However, there are more than 10,000 possible places to check and add. One can do a shortcut to add tags as follow:

i) Search for <ghgpolicy name="CO2"> which originally tags all technologies and replace with <ghgpolicy name="CO2"><ghgpolicy name="CO2_FOS"> in Main_User_Workspace/input/gcam-data-system/xlm folder. 

ii) Search for lines relevant to cement technologies and remove <ghgpolicy name="CO2_FOS"> tags.This is relatively easy tasks since there are only two input files related to cement production, namely cement.xml and cement_incelas_gcam3.xml

iii) We do not need to remove <ghgpolicy name="CO2_FOS"> tags for bio-related emission as GCAM assumes net zero.
Once all tags in   Main_User_Workspace/input/gcam-data-system/xlm folder, we can evoke this fossil fuel accounting by replacing <ghgpolicy name="CO2"> tags with <ghgpolicy name="CO2_FOS"> tags in policy xml files in   Main_User_Workspace/input/policy folder. Then one can run the model as usual.

In order include total CO2 emissions in account of CO2 policy, there is no need to construct a new policy xml tag. However, for price-based policy, one needs to add a call to extra policy file by adding a line <Value name = "policy">../input/policy/global_uct.xml</Value> in <ScenarioComponents> of the file Main_User_Workspace/exe/configuration.xml. For quantity-based policy one needs to add a call to another extra policy file by adding a line  <Value name = "policy">../input/policy/global_uct_in_constraint.xml</Value> at the same location.

$\underline{\text{MERGE}}$: 

As addressed earlier, in MERGE available options, there is no option to account CO2 emissions from fossil fuels. We need to modify the main file Merge7/m7.gms to enforce CO2 emissions from fossils in accounting as follow: 

In a price-based policy, in order to include CO2 emissions from fossil fuels in account of CO2 policy, one needs to go to tax calculation equation, remove two terms that contributes to tax from total CO2 emissions (CLEV and CCEM), and replace it with a tax term for CO2 emissions from fossil fuels:  ghgtax("co2",tp,rg,sw)*CFOS(tp,rg,sw)

In a quantity-based policy, in order to include CO2 emissions from fossil fuels, one need to go to exogenous constraint on carbon emission (clevbd) and replace variables for  total CO2 emissions with variables for CO2 emissions from fossil fuels CFOS. 
